## Things to do:
#http://aiddata.org/country-level-research-datasets
1. Country to Country: sankey diagram with (a) sector in the middle (b) umbrella in the middle
2. *Liars*:  disbursed - committed (color shows # of failures): square tree 
3. *Corrupts*: total project cost- disbursed (color shows # of failures): square tree [do it by filtering for projects that were completed, i.e. start_date and end_date not zero]
4. **Association rule mining** on categorical variables associated with Nepal. (Present as word cloud with importance based on lift)
5. Pick some variables and predict whether a project is completed or not. rank the variables by importance. 

In [205]:
#load aid data, downloaded from http://aiddata.org/country-level-research-datasets
import pandas as pd
df=pd.read_csv('/Users/sumangiri/Downloads/aiddata2-1_research_release.csv')

In [206]:
#get rid of nulls
df=df.fillna(0)
df.head(2)

,aiddata_id,year,commitment_date,start_date,end_date,donor,donor_iso2,donor_iso3,umbrella,donor_project_id,...,investment_project,aiddata_purpose_code,aiddata_purpose_name,aiddata_activity_code,aiddata_activity_name,aiddata_feasibility_study,aiddata_technical_assistance,coalesced_purpose_code,coalesced_purpose_name,aiddata_is_conflict
0,21235774,2004,0,0,0,Belgium,BE,BEL,0,1840171,...,0,0,0,0,0,0,0,92010,Support to national ngos,0
1,94726192,2010,0,2010-06-07 00:00:00,2010-12-07 00:00:00,United Kingdom,GB,GBR,0,201848-101,...,0,0,0,0,0,0,0,72010,Emergency/distress relief,0


In [358]:
filter(lambda x: 'am' in x, df.columns)

['commitment_amount',
 'commitment_amount_currency',
 'commitment_amount_usd_nominal',
 'commitment_amount_usd_constant',
 'disbursement_amount',
 'disbursement_amount_currency',
 'disbursement_amount_usd_nominal',
 'disbursement_amount_usd_constant',
 'crs_flow_name',
 'untied_amount_usd_nominal',
 'partial_tied_amount_usd_nominal',
 'tied_amount_usd_nominal',
 'received_amount_usd_nominal',
 'irtc_amount_usd_nominal',
 'expert_commitment_amount_usd_nominal',
 'expert_extended_amount_usd_nominal',
 'export_credit_amount_usd_nominal',
 'outstanding_amount_usd_nominal',
 'arrears_principal_amount_usd_nominal',
 'arrears_interest_amount_usd_nominal',
 'future_ds_principal_amount_usd_nominal',
 'future_ds_interest_amount_usd_nominal',
 'crs_purpose_name',
 'sector_programme',
 'aiddata_purpose_name',
 'aiddata_activity_name',
 'coalesced_purpose_name']

In [207]:
#Change sector to easily digestible format
mapping={"X. Support to NGO's":'NGO support', 'VIII.1. Emergency Response':'Emergency',
       'I.5.a. Government & Civil Society-general':'Government',
       'I.5.b. Conflict, Peace & Security':'Peace',
       'II.4. Banking & Financial Services':'Banking', 'IV.2. Other Multisector':'Multisector', 0:'Other',
       'I.6. Other Social Infrastructure & Services':'Infrastructure',
       'I.1.a. Education, Level Unspecified':'Education', 'I.1.b. Basic Education':'Education',
       'II.5. Business & Other Services':'Business',
       'I.1.d. Post-Secondary Education':'Education', 'I.2.a. Health, General':'Health',
       'IX. Administrative Costs of Donors':'Administrative', 'III.1.a. Agriculture':'Agriculture',
       'I.4. Water Supply & Sanitation':'Water', 'III.1.b. Forestry':'Forestry',
       'XII. Unallocated / Unspecified':'Unspecified', 'III.1.c. Fishing':'Fishing',
       'II.2. Communications':'Communications', 'III.3.a. Trade Policies & Regulations':'Trade',
       'I.3. Population Pol./Progr. & Reproductive Health':'Health',
       'I.2.b. Basic Health':'Health', 'III.2.a. Industry':'Industry',
       'I.1.c. Secondary Education':'Education',
       'VI.2. Dev. Food Aid/Food Security Ass.':'Food Security', 'II.3. Energy':'Energy',
       'IV.1. General Environment Protection':'Environment', 'II.1. Transport & Storage':'Transportation',
       'III.3.b. Tourism':'Tourism', 'VI.1. General Budget Support':'Budget',
       'VII. Action Relating to Debt':'Debt',
       'VIII.2. Reconstruction Relief & Rehabilitation':'Reconstruction',
       'VIII.3. Disaster Prevention & Preparedness':'Disaster Prevention',
       'III.2.b. Mineral Resources & Mining':'Mineral Resources',
       'XI. Refugees in Donor Countries':'Refugees', 'VI.3. Other Commodity Ass.':'Commodity',
       'III.2.c. Construction':'Construction'}

In [208]:
df.sector=df.sector.map(mapping)

In [209]:
df=df[df.aiddata_id!=2445044] #appears to have flawed amount numbers

In [210]:
#sanity check
df.commitment_amount_usd_nominal.sum()/1e12, \
df.disbursement_amount_usd_nominal.sum()/1e12,\
df.total_project_cost_usd_nominal.sum()/1e12

(4.05976345157199, 0.9828183649235868, 2.800125861167532)

In [211]:
#Select features of interest
df1=df[['donor',\
'recipient',\
'sector',\
'disbursement_amount_usd_nominal',\
'commitment_amount_usd_nominal',\
'total_project_cost_usd_nominal']]

In [212]:
df1.head(2)

,donor,recipient,sector,disbursement_amount_usd_nominal,commitment_amount_usd_nominal,total_project_cost_usd_nominal
0,Belgium,El Salvador,NGO support,254814.262641,254814,0
1,United Kingdom,Guatemala,Emergency,154440.154440,154440,0


In [213]:
amt_feature='commitment_amount_usd_nominal'

In [214]:
#get donor to sector
d2s=df1[['donor','sector',amt_feature]]\
.groupby(['donor','sector']).sum().reset_index().\
sort_values(amt_feature,ascending=[0])
d2s.columns=['source','target','weight']

In [215]:
#get sector to recipient
s2r=df1[['sector','recipient',amt_feature]]\
.groupby(['recipient','sector']).sum().reset_index().\
sort_values(amt_feature,ascending=[0])
s2r=s2r[['sector','recipient',amt_feature]]
s2r.columns=['source','target','weight']

### Top 5 donors to top 10 recipients through top 5 sectors
for cases with more than USD 100 million disbursed

In [216]:
#get top 5 sectors (Exclude other which is the number 1 sector)
sct1= list(df1[[amt_feature,'sector']].groupby(['sector']).sum()\
.reset_index().sort_values(amt_feature,ascending=[0])\
.sector[1:8])

In [217]:
#take donors and recipients only as long as they are in the sectors of interest
d2s_subset=d2s[d2s.target.isin(sct1)]
s2r_subset=s2r[s2r.source.isin(sct1)]

In [218]:
#how top 5 donors give aid
#how top 15 recipients receive aid
rcp1=s2r_subset.groupby('target').sum().reset_index().sort_values('weight',ascending=[0]).target[0:10]
dnr1=d2s_subset.groupby('source').sum().reset_index().sort_values('weight',ascending=[0]).source[0:5]

In [219]:
#take donors and recipients only as long as they are top 10 or top 5
d2s_subset1=d2s_subset[d2s_subset.source.isin(dnr1)]
s2r_subset1=s2r_subset[s2r_subset.target.isin(rcp1)]

In [220]:
df_final=pd.concat([d2s_subset1,s2r_subset1],0).sort_values('weight',ascending=[0])

In [221]:
nodes=pd.DataFrame(pd.concat([df_final.source,df_final.target],0).unique(), columns=['id'])
idx=range(nodes.shape[0])
dc=dict()
#create a dictionary to establish a map
for i in idx:
    dc[nodes.iloc[i].id]=i

In [222]:
df_final.source=df_final.source.map(dc)
df_final.target=df_final.target.map(dc)

In [223]:
#Get rid of data where aid is low
df_final=df_final[df_final.weight > 1e8]

In [224]:
#Convert to billion and save file
df_final.weight=df_final.weight*1.0/1e9
df_final.to_csv('links.csv')
nodes.to_csv('nodes.csv')

### Aid flow to Nepal 
(for cases with more than 10 million USD disbursed)

In [225]:
country_list=['Nepal', 'Sri Lanka']
# Get the sectors when recipient is Nepal
df_nep_temp=pd.concat([d2s,s2r],0).sort_values('weight',ascending=[0])
s2r_nep=df_nep_temp[df_nep_temp.target.isin(country_list)]
s2r_nep=s2r_nep[s2r_nep.weight>1e7]

In [226]:
#get donor to sector for when recipient is Nepal
d2s_nep=df1[['donor','sector',amt_feature]]\
[df1.recipient.isin(country_list)]\
.groupby(['donor','sector']).sum().reset_index().\
sort_values(amt_feature,ascending=[0])
d2s_nep.columns=['source','target','weight']
d2s_nep=d2s_nep[d2s_nep.weight>1e7]

In [227]:
#get the top "n" sectors
sct_nep=pd.concat([d2s_nep[['target','weight']],s2r_nep[['source','weight']].\
                   rename(columns={'source': 'target'})],0).\
groupby('target').sum().reset_index().sort_values('weight',ascending=[0]).target[0:10]

In [228]:
d2s_nep_subset=d2s_nep[d2s_nep.target.isin(sct_nep)]
s2r_nep_subset=s2r_nep[s2r_nep.source.isin(sct_nep)]

In [229]:
#take top 15 donors only
dnr_nep=d2s_nep_subset.groupby('source').sum().reset_index().sort_values('weight',ascending=[0]).source[0:20]
d2s_nep_subset1=d2s_nep_subset[d2s_nep_subset.source.isin(dnr_nep)]

In [230]:
df_nep=pd.concat([d2s_nep_subset1,s2r_nep_subset],0).sort_values('weight',ascending=[0])

In [231]:
nodes_nep=pd.DataFrame(pd.concat([df_nep.source,df_nep.target],0).unique(), columns=['id'])
idx_nep=range(nodes_nep.shape[0])
dc_nep=dict()
#create a dictionary to establish a map
for i in idx_nep:
    dc_nep[nodes_nep.iloc[i].id]=i

In [232]:
df_nep.source=df_nep.source.map(dc_nep)
df_nep.target=df_nep.target.map(dc_nep)

In [233]:
df_nep=df_nep[df_nep.weight > 1e7]

In [234]:
#Convert to billion and save file
df_nep.weight=df_nep.weight*1.0/1e6
df_nep.to_csv('links_nep.csv')
nodes_nep.to_csv('nodes_nep.csv')

In [235]:
#sanity check (it should be zero if 1 million threshold on s2r_nep and d2s_nep is not applied)
#s2r_nep.weight.sum()-d2s_nep.weight.sum()

In [353]:
from IPython.display import IFrame
IFrame('sankey_aid_data_2016.html',width=750,height=550, scroll=False,display='block',lineheight=0)

In [355]:
IFrame('sankey_aid_data_nep_2016.html',width=750,height=550, scroll=False,display='block',lineheight=0)

In [356]:
IFrame('sankey_aid_data.html',width=750,height=550, scroll=False,display='block',lineheight=0)

In [357]:
IFrame('sankey_aid_data_nep.html',width=750,height=550, scroll=False,display='block',lineheight=0)

In [376]:
IFrame('treemap_dis_pc.html',width=500,height=540, scroll=False,display='block',lineheight=0)

In [379]:
IFrame('treemap_com_dis.html',width=600,height=640, scroll=False,display='block',lineheight=0)